# Proyecto A: Optimización en la Planeación de Transporte Vehicular Urbana Para LogistiCo

## Integrantes

- Barrera Toro, Javier Steven
- Rolon Toloza, Julian Santiago

# Implementación del modelo matemático y procesamiento de datos

In [2]:
import pandas as pd
from geopy.distance import geodesic
from pyomo.environ import *
from pyomo.opt import SolverFactory

## Procesamiento de datos

In [3]:
import pandas as pd
from geopy.distance import geodesic
import os

ruta_clientes_csv = 'caso_base/clients.csv'
ruta_depositos_csv = 'caso_base/depots.csv'
df_vehiculos = pd.read_csv('caso_base/vehicles.csv')

directorio_salida_consolidado = 'Proyecto_Caso_Base'
nombre_archivo_ubicaciones_consolidadas = 'locations.csv'
ruta_ubicaciones_consolidadas_csv = os.path.join(directorio_salida_consolidado, nombre_archivo_ubicaciones_consolidadas)

if not os.path.exists(directorio_salida_consolidado):
    os.makedirs(directorio_salida_consolidado)
    print(f"Directorio '{directorio_salida_consolidado}' creado.")

try:
    print(f"Cargando datos de clientes desde: {ruta_clientes_csv}")
    df_clientes = pd.read_csv(ruta_clientes_csv)

    print(f"Cargando datos de depósitos desde: {ruta_depositos_csv}")
    df_depositos = pd.read_csv(ruta_depositos_csv)

except FileNotFoundError as e:
    print(f"Error: No se pudo encontrar el archivo {e.filename}. Asegúrate de que las rutas sean correctas.")
    exit()

df_depositos_ubic = df_depositos[['LocationID', 'Longitude', 'Latitude']].copy()
df_clientes_ubic = df_clientes[['LocationID', 'Longitude', 'Latitude']].copy()

ubicaciones_consolidadas_df = pd.concat([
    df_depositos_ubic,
    df_clientes_ubic
], ignore_index=True)

try:
    ubicaciones_consolidadas_df.to_csv(ruta_ubicaciones_consolidadas_csv, index=False)
    print(f"Archivo de ubicaciones consolidadas guardado en: {ruta_ubicaciones_consolidadas_csv}")
except Exception as e:
    print(f"Error al guardar el archivo de ubicaciones consolidadas: {e}")
    exit()

try:
    print(f"Cargando ubicaciones consolidadas desde: {ruta_ubicaciones_consolidadas_csv}")
    df_ubicaciones_para_matriz = pd.read_csv(ruta_ubicaciones_consolidadas_csv)
except FileNotFoundError:
    print(f"Error: El archivo consolidado '{ruta_ubicaciones_consolidadas_csv}' no fue encontrado.")
    exit()

print("Calculando matriz de distancias...")
lista_ids_ubicaciones = df_ubicaciones_para_matriz['LocationID'].tolist()
n_ubicaciones = len(lista_ids_ubicaciones)
matriz_distancias_lista = [[0.0] * n_ubicaciones for _ in range(n_ubicaciones)]

for i in range(n_ubicaciones):
    fila_origen = df_ubicaciones_para_matriz.iloc[i]
    coord_origen = (fila_origen['Latitude'], fila_origen['Longitude'])
    for j in range(n_ubicaciones):
        if i == j:
            matriz_distancias_lista[i][j] = 0.0
        else:
            fila_dest = df_ubicaciones_para_matriz.iloc[j]
            coord_dest = (fila_dest['Latitude'], fila_dest['Longitude'])
            distancia_km = geodesic(coord_origen, coord_dest).kilometers
            matriz_distancias_lista[i][j] = distancia_km

print("Matriz de distancias calculada.")

df_matriz_distancias = pd.DataFrame(
    matriz_distancias_lista,
    index=lista_ids_ubicaciones,
    columns=lista_ids_ubicaciones
)

print("\nMatriz de Distancias (DataFrame):")
print(df_matriz_distancias)

Cargando datos de clientes desde: caso_base/clients.csv
Cargando datos de depósitos desde: caso_base/depots.csv
Archivo de ubicaciones consolidadas guardado en: Proyecto_Caso_Base\locations.csv
Cargando ubicaciones consolidadas desde: Proyecto_Caso_Base\locations.csv
Calculando matriz de distancias...
Matriz de distancias calculada.

Matriz de Distancias (DataFrame):
           1          2          3          4          5          6   \
1    0.000000  17.182367  10.608396   6.370334  16.521196  10.565755   
2   17.182367   0.000000  10.270212  12.367369   0.803686   9.184090   
3   10.608396  10.270212   0.000000   4.239027   9.466781  10.798470   
4    6.370334  12.367369   4.239027   0.000000  11.603070   9.291240   
5   16.521196   0.803686   9.466781  11.603070   0.000000   8.884619   
6   10.565755   9.184090  10.798470   9.291240   8.884619   0.000000   
7    9.686597   7.501926   5.769840   5.454592   6.834748   5.089974   
8   15.435533  13.921205   6.024955   9.518335  13.194

In [ ]:
from pyomo.environ import *
import pandas as pd
import os

N_ids = df_ubicaciones_para_matriz['LocationID'].tolist()
C_ids = df_clientes['LocationID'].tolist()
D_ids = df_depositos['LocationID'].tolist()
V_ids = df_vehiculos['VehicleID'].tolist()

if not D_ids:
    print("Error: No hay depósitos definidos.")
    exit()
deposito_principal_id = D_ids[0]

demanda_param = pd.Series(df_clientes['Demand'].values, index=df_clientes['LocationID']).to_dict()
capacidad_param = pd.Series(df_vehiculos['Capacity'].values, index=df_vehiculos['VehicleID']).to_dict()
autonomia_param = pd.Series(df_vehiculos['Range'].values, index=df_vehiculos['VehicleID']).to_dict()
dist_param = df_matriz_distancias.stack().to_dict()

modelo = ConcreteModel()

modelo.N_set = Set(initialize=N_ids)
modelo.C_set = Set(initialize=C_ids)
modelo.V_set = Set(initialize=V_ids)
modelo.D_set = Set(initialize=D_ids)

modelo.demanda = Param(modelo.C_set, initialize=demanda_param, default=0)
modelo.capacidad = Param(modelo.V_set, initialize=capacidad_param, default=0)
modelo.autonomia = Param(modelo.V_set, initialize=autonomia_param, default=0)
modelo.dist = Param(modelo.N_set, modelo.N_set, initialize=dist_param, default=999999)

modelo.x = Var(modelo.N_set, modelo.N_set, modelo.V_set, domain=Binary)
num_clientes_para_mtz = len(modelo.C_set)
modelo.u = Var(modelo.C_set, modelo.V_set, bounds=(1, num_clientes_para_mtz if num_clientes_para_mtz > 0 else 1), domain=Integers)

def objetivo_expr(m):
    return sum(m.dist[i,j] * m.x[i,j,k]
               for i in m.N_set for j in m.N_set for k in m.V_set if i != j)
modelo.obj = Objective(rule=objetivo_expr, sense=minimize)

modelo.rest_no_self_loops = ConstraintList()
for k in modelo.V_set:
    for i in modelo.N_set:
        modelo.rest_no_self_loops.add(modelo.x[i,i,k] == 0)

modelo.rest_visita_unica = ConstraintList()
for j_cliente in modelo.C_set:
    modelo.rest_visita_unica.add(
        sum(modelo.x[i,j_cliente,k] for i in modelo.N_set if i != j_cliente for k in modelo.V_set) == 1
    )

modelo.rest_salida_deposito = ConstraintList()
for k in modelo.V_set:
    modelo.rest_salida_deposito.add(
        sum(modelo.x[deposito_principal_id,j,k] for j in modelo.N_set if j != deposito_principal_id) <= 1
    )

modelo.rest_entrada_deposito = ConstraintList()
for k in modelo.V_set:
    modelo.rest_entrada_deposito.add(
        sum(modelo.x[i,deposito_principal_id,k] for i in modelo.N_set if i != deposito_principal_id) <= 1
    )

modelo.rest_salida_regreso_consistencia = ConstraintList()
for k in modelo.V_set:
    modelo.rest_salida_regreso_consistencia.add(
        sum(modelo.x[deposito_principal_id,j,k] for j in modelo.N_set if j != deposito_principal_id) ==
        sum(modelo.x[i,deposito_principal_id,k] for i in modelo.N_set if i != deposito_principal_id)
    )

modelo.rest_flujo = ConstraintList()
for k in modelo.V_set:
    for nodo_cliente in modelo.C_set:
        sum_entradas = sum(modelo.x[i,nodo_cliente,k] for i in modelo.N_set if i != nodo_cliente)
        sum_salidas = sum(modelo.x[nodo_cliente,j,k] for j in modelo.N_set if j != nodo_cliente)
        modelo.rest_flujo.add(sum_entradas == sum_salidas)

M_mtz = len(modelo.C_set) + 1 if len(modelo.C_set) > 0 else 2
modelo.rest_mtz = ConstraintList()
if len(modelo.C_set) > 1 :
    for k in modelo.V_set:
        for i_cliente in modelo.C_set:
            for j_cliente in modelo.C_set:
                if i_cliente != j_cliente:
                    modelo.rest_mtz.add(
                        modelo.u[i_cliente,k] - modelo.u[j_cliente,k] + M_mtz * modelo.x[i_cliente,j_cliente,k] <= M_mtz - 1
                    )

modelo.rest_capacidad = ConstraintList()
for k in modelo.V_set:
    modelo.rest_capacidad.add(
        sum(modelo.demanda[j_cliente] * sum(modelo.x[i,j_cliente,k] for i in modelo.N_set if i != j_cliente)
            for j_cliente in modelo.C_set) <= modelo.capacidad[k]
    )

modelo.rest_autonomia = ConstraintList()
for k in modelo.V_set:
    modelo.rest_autonomia.add(
        sum(modelo.dist[i,j] * modelo.x[i,j,k] for i in modelo.N_set for j in modelo.N_set if i != j) <= modelo.autonomia[k]
    )

solver = SolverFactory('glpk')
solver.options['tmlim'] = 300 # Límite de tiempo en segundos (300 seg = 5 minutos)
# solver.options['mipgap'] = 0.05 # Para algunos solvers, establece un gap de 5% (GLPK usa --mipgap en línea de comando)

resultados = solver.solve(modelo, tee=False)

if (resultados.solver.status == SolverStatus.ok) and \
   (resultados.solver.termination_condition == TerminationCondition.optimal or
    resultados.solver.termination_condition == TerminationCondition.feasible or # Si tmlim se alcanza y hay solución
    resultados.solver.termination_condition == TerminationCondition.maxTimeLimit): # Otra forma de indicar límite de tiempo
    print(f"Valor de la Función Objetivo (Distancia Total): {value(modelo.obj):.2f}")

    for k_vehiculo_id in modelo.V_set:
        vehiculo_usado_flag = any(value(modelo.x[deposito_principal_id, j_nodo_id, k_vehiculo_id]) > 0.5
                                  for j_nodo_id in modelo.N_set if j_nodo_id != deposito_principal_id)

        if not vehiculo_usado_flag:
            # No imprimir nada si el vehículo no se usó, para reducir la salida
            continue

        print(f"\nVehículo {k_vehiculo_id}:")
        ruta_actual_str = str(deposito_principal_id) 
        nodo_actual_id = deposito_principal_id
        distancia_vehiculo = 0
        demanda_atendida_vehiculo = 0
        nodos_visitados_en_ruta = {deposito_principal_id}

        for _ in range(len(modelo.N_set)):
            siguiente_nodo_encontrado = False
            for j_nodo_id in modelo.N_set:
                # Comprobar si el valor de x es suficientemente cercano a 1
                if j_nodo_id != nodo_actual_id and value(modelo.x[nodo_actual_id, j_nodo_id, k_vehiculo_id]) > 0.5:
                    if j_nodo_id in nodos_visitados_en_ruta and j_nodo_id != deposito_principal_id:
                        continue

                    ruta_actual_str += f" -> {str(j_nodo_id)}"
                    distancia_vehiculo += value(modelo.dist[nodo_actual_id, j_nodo_id])
                    if j_nodo_id in modelo.C_set: # Solo sumar demanda si es un cliente
                        demanda_atendida_vehiculo += value(modelo.demanda[j_nodo_id])
                    
                    nodo_actual_id = j_nodo_id
                    nodos_visitados_en_ruta.add(nodo_actual_id)
                    siguiente_nodo_encontrado = True
                    break

            if not siguiente_nodo_encontrado or nodo_actual_id == deposito_principal_id :
                # Asegurarse de que si la ruta no terminó explícitamente en el depósito pero debería, se imprima el regreso
                if nodo_actual_id != deposito_principal_id and any(value(modelo.x[nodo_actual_id, deposito_principal_id, k_vehiculo_id]) > 0.5 for _i in [0]):
                     ruta_actual_str += f" -> {deposito_principal_id}"
                     distancia_vehiculo += value(modelo.dist[nodo_actual_id, deposito_principal_id])
                break
        
        print(f"  Ruta: {ruta_actual_str}")
        print(f"  Distancia Recorrida: {distancia_vehiculo:.2f} (Autonomía: {value(modelo.autonomia[k_vehiculo_id])})")
        print(f"  Demanda Atendida: {demanda_atendida_vehiculo} (Capacidad: {value(modelo.capacidad[k_vehiculo_id])})")

elif resultados.solver.status == SolverStatus.warning and \
     resultados.solver.termination_condition == TerminationCondition.maxTimeLimit:
    print("\nLímite de Tiempo Alcanzado.")
    # Intentar imprimir el valor objetivo si existe una solución factible
    try:
        obj_val = value(modelo.obj)
        print(f"Valor de la Función Objetivo (Distancia Total) en el límite de tiempo: {obj_val:.2f}")
        # Aquí también podrías intentar imprimir la solución parcial como en el bloque 'if' anterior
    except Exception:
        print("No se pudo obtener el valor del objetivo (puede que no haya solución factible aún).")

else:
    print("\nNo se encontró una solución óptima o factible dentro de los criterios.")
    print(f"Estado del Solver: {resultados.solver.status}")
    print(f"Condición de Terminación: {resultados.solver.termination_condition}")

Valor de la Función Objetivo (Distancia Total): 178.81

Vehículo 1:
  Ruta: 1 -> 7 -> 2 -> 17 -> 11 -> 25 -> 18 -> 5 -> 16 -> 1
  Distancia Recorrida: 53.37 (Autonomía: 170)
  Demanda Atendida: 123 (Capacidad: 130)

Vehículo 2:
  Ruta: 1 -> 24 -> 4 -> 15 -> 13 -> 21 -> 1
  Distancia Recorrida: 19.50 (Autonomía: 200)
  Demanda Atendida: 69 (Capacidad: 140)

Vehículo 4:
  Ruta: 1 -> 14 -> 19 -> 12 -> 10 -> 23 -> 20 -> 1
  Distancia Recorrida: 57.95 (Autonomía: 90)
  Demanda Atendida: 99 (Capacidad: 100)

Vehículo 7:
  Ruta: 1 -> 9 -> 6 -> 3 -> 8 -> 22 -> 1
  Distancia Recorrida: 47.99 (Autonomía: 150)
  Demanda Atendida: 86 (Capacidad: 110)


# Casos

## Caso base

## Caso a

## Caso b

# Análisis de resultados y visualizaciones